<a href="https://colab.research.google.com/github/leticiaschaves/leticiaschaves/blob/data-projects/LESSON_02_Pyspark_An%C3%A1lise_explorat%C3%B3ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise exploratória com Pyspark

In [ ]:
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 40.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=4f39467f026b5b5946a196d0c4a5e3ada9a21f241df8d389f7b9def34cb587ab
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


### Importe aqui as bibliotecas necessárias

In [ ]:
from pyspark.sql import SparkSession

import pyspark.sql.functions as F



### Inicie uma sessão spark

In [ ]:
spark = SparkSession.builder.appName('teste').getOrCreate()

### Baixe os dados deste [endereço](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv) e carregue-os como um *dataset* em uma variável.

In [ ]:
df_chipotle = spark.read.csv('/content/chipotle.tsv', header=True, sep='\t')

### Mostre as 10 primeiras linhas

In [ ]:
df_chipotle.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

### Quantas observações/informações existem no total?

In [ ]:
df_chipotle.count()

4622

### Qual o número de colunas no dataset?

In [ ]:
totalColumns = len(df_chipotle.columns)

totalColumns

5

### Imprima o nome de todas as colunas

In [ ]:
df_chipotle.columns

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

## groupBy:
Groups the DataFrame using the specified columns, so we can run aggregation on them. See GroupedData for all the available aggregate functions.

groupby() is an alias for groupBy().

New in version 1.3.0.

### Parameters
colslist, str or Column
columns to group by. Each element should be a column name (string) or an expression (Column).

## orderBy:
Returns a new DataFrame sorted by the specified column(s).

New in version 1.3.0.

### Parameters
cols: *str, list, or Column, optional*
list of Column or column names to sort by.

### Other Parameters
ascending: *bool or list, optional*

boolean or list of boolean (default True). Sort ascending vs. descending. Specify list for multiple sort orders. If a list is specified, length of the list must equal length of the cols.

### Qual o item mais pedido? Quantos foram pedidos no total?

A função orderBy cria uma nova coluna 'count' já ordenado na decrescente

In [ ]:
df_chipotle.groupBy('item_name').count().orderBy('count', ascending = False).show(1)

+------------+-----+
|   item_name|count|
+------------+-----+
|Chicken Bowl|  726|
+------------+-----+
only showing top 1 row



In [ ]:
# ajuda através da documentação
help (df_chipotle.orderBy)

Help on method sort in module pyspark.sql.dataframe:

sort(*cols: Union[str, pyspark.sql.column.Column, List[Union[str, pyspark.sql.column.Column]]], **kwargs: Any) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` sorted by the specified column(s).
    
    .. versionadded:: 1.3.0
    
    Parameters
    ----------
    cols : str, list, or :class:`Column`, optional
         list of :class:`Column` or column names to sort by.
    
    Other Parameters
    ----------------
    ascending : bool or list, optional
        boolean or list of boolean (default ``True``).
        Sort ascending vs. descending. Specify list for multiple sort orders.
        If a list is specified, length of the list must equal length of the `cols`.
    
    Examples
    --------
    >>> df.sort(df.age.desc()).collect()
    [Row(age=5, name='Bob'), Row(age=2, name='Alice')]
    >>> df.sort("age", ascending=False).collect()
    [Row(age=5, name='Bob'), Row(age=

### Quantos itens em geral foram pedidos?

In [ ]:
df_chipotle.agg(F.sum(F.col('quantity'))).show()

+-------------+
|sum(quantity)|
+-------------+
|       4972.0|
+-------------+



### Converta a coluna preço para float

Obs: Tirar o cifrão e converter para float

In [ ]:
df_chipotle.show()

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
|       5|       1| Chips and Guacamole|           

In [ ]:
def corrigir_nomes(nome):
    nome = nome.replace('$', '')
    return nome

In [ ]:
from pyspark.sql.types import StringType

In [ ]:
novaFuncao = F.udf(corrigir_nomes, StringType())

In [ ]:
df_chipotle.withColumn('item_price',novaFuncao(F.col('item_price')).cast('float')).printSchema()

root
 |-- order_id: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: float (nullable = true)



In [ ]:
df = df_chipotle.withColumn('item_price',novaFuncao(F.col('item_price')).cast('float'))

### Qual foi a entrada em caixa total do dataset inteiro?

In [ ]:
df.select(F.sum('item_price')).show()

+------------------+
|   sum(item_price)|
+------------------+
|34500.159754514694|
+------------------+



### Qual a entrada média em caixa por pedido?

In [ ]:
df.select(F.avg('item_price')).show()

+-----------------+
|  avg(item_price)|
+-----------------+
|7.464335732261942|
+-----------------+



### Quantos item diferentes são vendidos?

In [ ]:
help (df.dropDuplicates)

Help on method dropDuplicates in module pyspark.sql.dataframe:

dropDuplicates(subset: Union[List[str], NoneType] = None) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Return a new :class:`DataFrame` with duplicate rows removed,
    optionally only considering certain columns.
    
    For a static batch :class:`DataFrame`, it just drops duplicate rows. For a streaming
    :class:`DataFrame`, it will keep all data across triggers as intermediate state to drop
    duplicates rows. You can use :func:`withWatermark` to limit how late the duplicate data can
    be and system will accordingly limit the state. In addition, too late data older than
    watermark will be dropped to avoid any possibility of duplicates.
    
    :func:`drop_duplicates` is an alias for :func:`dropDuplicates`.
    
    .. versionadded:: 1.4.0
    
    Examples
    --------
    >>> from pyspark.sql import Row
    >>> df = sc.parallelize([ \
    ...     Row(name='Alice', age=5, height=80), \


In [ ]:
df.dropDuplicates(['item_name']).count()

50

### Quantos produtos custam mais que $10,00?

In [ ]:
df.filter(df['item_price']>10).count()

1130

### Qual é o preço de cada item?
(criar um dataframe com o nome do item e seu respectivo preço)

In [ ]:
df.dropDuplicates(['item_name']).select(['item_name','item_price']).show()

+--------------------+----------+
|           item_name|item_price|
+--------------------+----------+
| Carnitas Soft Tacos|      9.25|
|  Chicken Soft Tacos|      8.75|
|               Salad|       7.4|
|         Steak Salad|      8.99|
|                Bowl|      22.2|
| Veggie Crispy Tacos|      8.49|
|   6 Pack Soft Drink|      6.49|
|Chicken Crispy Tacos|      8.75|
|   Veggie Soft Tacos|     11.25|
| Carnitas Salad Bowl|     11.89|
|     Chicken Burrito|      8.49|
|   Veggie Salad Bowl|     11.25|
|         Veggie Bowl|     11.25|
|       Steak Burrito|     11.75|
|  Steak Crispy Tacos|      9.25|
|   Canned Soft Drink|      1.25|
|        Chicken Bowl|     16.98|
|       Side of Chips|      1.69|
|                Izze|      3.39|
|  Chicken Salad Bowl|      8.75|
+--------------------+----------+
only showing top 20 rows



### Ordene as linhas por ordem alfabética de nome

In [ ]:
df_chipotle.orderBy('item_name').show()

+--------+--------+-----------------+------------------+----------+
|order_id|quantity|        item_name|choice_description|item_price|
+--------+--------+-----------------+------------------+----------+
|     511|       1|6 Pack Soft Drink|            [Coke]|    $6.49 |
|    1253|       1|6 Pack Soft Drink|        [Lemonade]|    $6.49 |
|     520|       1|6 Pack Soft Drink|          [Sprite]|    $6.49 |
|     148|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |
|     566|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |
|     168|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |
|     708|       1|6 Pack Soft Drink|            [Coke]|    $6.49 |
|     230|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |
|     709|       1|6 Pack Soft Drink|       [Diet Coke]|    $6.49 |
|     298|       1|6 Pack Soft Drink|          [Nestea]|    $6.49 |
|     749|       1|6 Pack Soft Drink|            [Coke]|    $6.49 |
|     363|       1|6 Pack Soft Drink|           

### Quantas vezes o item mais pedido foi vendido?

In [ ]:
df.groupBy('item_name').count().orderBy('count', ascending = False).show(1)

+------------+-----+
|   item_name|count|
+------------+-----+
|Chicken Bowl|  726|
+------------+-----+
only showing top 1 row



### Quantas vezes o item "Veggie Salad Bowl" foi vendido?

In [ ]:
df.groupBy('item_name').agg(F.sum('quantity')).filter(F.col('item_name') == 'Veggie Salad Bowl').show()

+-----------------+-------------+
|        item_name|sum(quantity)|
+-----------------+-------------+
|Veggie Salad Bowl|         18.0|
+-----------------+-------------+



### Quantas vezes alguém pediu *mais de um* item "Canned Soda"?

In [ ]:
df.filter((F.col('item_name') == 'Canned Soda') & (F.col('quantity') > 1)).count()

20